# Data Acquisition
---

In [1]:
from bs4 import BeautifulSoup as bs
import datetime
import pandas as pd
import pickle
import re
import requests
# from selenium import webdriver
import sqlite3
import time as time_module

In [2]:
press_list = ['daum', 'naver',
              'seoulilbo', 'dtoday', 'asiailbo', 'labortoday', 'm-i',
              'ekn', 'busan', 'imaeil', 'kookje', 'yeongnam']
press_dict = {'daum':'다음', 'naver':'네이버',
              'seoulilbo':'서울일보', 'dtoday':'일간투데이', 'asiailbo':'아시아일보',
              'labortoday':'매일노동뉴스', 'm-i':'매일일보', 'ekn':'에너지경제',
              'busan':'부산일보', 'imaeil':'매일신문', 'kookje':'국제신문',
              'yeongnam':'영남일보'}

daum_press_list = ['EBS', 'IT동아', 'JTBC', 'KBS', 'KTV',
                   'MBC', 'MBN', 'SBS', 'SBS CNBC', 'YTN',
                   'ZDNet Korea', 'bnt뉴스', '게임동아', '게임톡', '경향신문',
                   '국민일보', '기자협회보', '노컷뉴스', '뉴스1', '뉴시스',
                   '데일리안', '동아사이언스', '동아일보', '디지털타임스', '로이터',
                   '매경게임진', '매일경제', '머니S', '머니투데이', '문화일보',
                   '미디어오늘', '서울경제', '서울신문', '세계일보', '아시아경제',
                   '아이뉴스24', '연합뉴스', '연합뉴스TV', '오마이뉴스', '오토타임즈',
                   '이데일리', '전자신문', '조선비즈', '조선일보', '중앙일보',
                   '채널A', '코리아헤럴드', '쿠키뉴스', '파이낸셜뉴스', '포토친구',
                   '프레시안', '한겨레', '한국경제', '한국경제TV', '한국일보',
                   '헤럴드경제']
naver_press_list = ['JTBC', 'KBS', 'MBC', 'MBN', 'SBS',
                    'SBS CNBC', 'TV조선', 'YTN', 'ZDNet Korea', '강원일보',
                    '경향신문', '국민일보', '기자협회보', '노컷뉴스', '뉴스1',
                    '뉴시스', '데일리안', '동아사이언스', '동아일보', '디지털데일리',
                    '디지털타임스', '로이터', '매일경제', '매일신문', '머니S',
                    '머니투데이', '문화일보', '미디어오늘', '부산일보', '블로터',
                    '서울경제', '서울신문', '세계일보', '아시아경제', '아이뉴스24',
                    '여성신문', '연합뉴스', '연합뉴스TV', '오마이뉴스', '이데일리',
                    '일다', '전자신문', '조선비즈', '조선일보', '조세일보',
                    '중앙일보', '참세상', '채널A', '코리아헤럴드', '코메디닷컴',
                    '파이낸셜뉴스', '프레시안', '한겨레', '한국경제', '한국경제TV',
                    '한국일보', '헤럴드경제', '헬스조선']

section_list = ['society', 'politics', 'economic', 'culture', 'digital', 'global']
section_dict = {'society':'사회', 'politics':'정치', 'economic':'경제',
               'culture':'문화', 'digital':'IT', 'global':'세계'}

base_urls = {'daum':
             {'society':'http://media.daum.net/breakingnews/society',
              'politics':'http://media.daum.net/breakingnews/politics',
              'economic':'http://media.daum.net/breakingnews/economic',
              'culture':'http://media.daum.net/breakingnews/culture',
              'digital':'http://media.daum.net/breakingnews/digital',
              'global':'http://media.daum.net/breakingnews/foreign'
             },
             'seoulilbo':
             {'society':'http://www.seoulilbo.com/news/articleList.html?sc_section_code=S1N10&view_type=sm',
              'politics':'http://www.seoulilbo.com/news/articleList.html?sc_section_code=S1N8&view_type=sm',
              'economic':'http://www.seoulilbo.com/news/articleList.html?sc_section_code=S1N9&view_type=sm',
              'culture':'http://www.seoulilbo.com/news/articleList.html?sc_section_code=S1N11&view_type=sm',
              'digital':'',
              'global':''
             },
             'dtoday':
             {'society':'',
              'politics':'http://www.dtoday.co.kr/news/articleList.html?sc_section_code=S1N1&view_type=sm',
              'economic':'http://www.dtoday.co.kr/news/articleList.html?sc_section_code=S1N2&view_type=sm',
              'culture':'',
              'digital':'',
              'global':''
             },
             'asiailbo':
             {'society':'http://www.asiailbo.co.kr/etnews/?cid=21030000',
              'politics':'http://www.asiailbo.co.kr/etnews/?cid=21010000',
              'economic':'http://www.asiailbo.co.kr/etnews/?cid=21020000',
              'culture':'http://www.asiailbo.co.kr/etnews/?cid=21040000',
              'digital':'',
              'global':''
             },
             'labortoday':
             {'society':'http://www.labortoday.co.kr/news/articleList.html?sc_section_code=S1N3&view_type=sm',
              'politics':'http://www.labortoday.co.kr/news/articleList.html?sc_section_code=S1N2&view_type=sm',
              'economic':'',    # 정치, 경제
              'culture':'',
              'digital':'',
              'global':''
             },
             'm-i':
             {'society':'http://www.m-i.kr/news/articleList.html?sc_section_code=S1N3&view_type=sm',
              'politics':'http://www.m-i.kr/news/articleList.html?sc_section_code=S1N1&view_type=tm',
              'economic':'http://www.m-i.kr/news/articleList.html?sc_section_code=S1N2&view_type=sm',
              'culture':'http://www.m-i.kr/news/articleList.html?sc_section_code=S1N22&view_type=tm',
              'digital':'',
              'global':''
             },
             'ekn':
             {'society':'http://www.ekn.kr/news/section_list_all.html?sec_no=25',
              'politics':'',    # 정치, 사회
              'economic':'http://www.ekn.kr/news/section_list_all.html?sec_no=130',
              'culture':'',
              'digital':'',
              'global':''
             },
             'busan':
             {'society':'http://news20.busan.com/news/social.jsp',
              'politics':'http://news20.busan.com/news/politics.jsp',
              'economic':'http://news20.busan.com/EconomyAndOcean/econocean.jsp',
              'culture':'http://news20.busan.com/news/culture.jsp',
              'digital':'',
              'global':''
             },
             'imaeil':
             {'society':'http://news.imaeil.com/SocietyAll/',
              'politics':'http://news.imaeil.com/PoliticsAll/',
              'economic':'http://news.imaeil.com/EconomyAll/',
              'culture':'http://news.imaeil.com/CultureAll/',
              'digital':'',
              'global':'http://news.imaeil.com/InternationalAll/'
             },
             'kookje':
             {'society':'http://www.kookje.co.kr/sub.htm?code=0300&vHeadTitle=%BB%E7%C8%B8',
              'politics':'http://www.kookje.co.kr/sub.htm?code=0100&vHeadTitle=%C1%A4%C4%A1',
              'economic':'http://www.kookje.co.kr/sub.htm?code=0200&vHeadTitle=%B0%E6%C1%A6',
              'culture':'http://www.kookje.co.kr/sub.htm?code=0500&vHeadTitle=%B9%AE%C8%AD',
              'digital':'http://www.kookje.co.kr/sub.htm?code=0800&vHeadTitle=IT%B0%FA%C7%D0',
              'global':'http://www.kookje.co.kr/sub.htm?code=0400&vHeadTitle=%B1%B9%C1%A6'
             },
             'yeongnam':
             {'society':'http://www.yeongnam.com/mnews/newsview.do?mode=subMain&cId=04',
              'politics':'http://www.yeongnam.com/mnews/newsview.do?mode=subMain&cId=02',
              'economic':'http://www.yeongnam.com/mnews/newsview.do?mode=subMain&cId=03',
              'culture':'http://www.yeongnam.com/mnews/newsview.do?mode=subMain&cId=08',
              'digital':'',    # 교육, 과학
              'global':'http://www.yeongnam.com/mnews/newsview.do?mode=subMain&cId=06'
             }
            }

headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36"}

## 불용어 제거
---

In [3]:
# 기본 정규식
reg_ex = {'email': r'[a-zA-Z0-9.!#$%&\'*+/=?^_`{|}~-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+',
          'sb': r'\[[^\]]*\]',
          'pb': r'\<[^\>]*\>'}

# 매칭되면 기사를 삭제하는 태그, 단어 및 정규식
stop_title_sb = ['[포토]', '[프로필]', '[★화보]', '[게시판]', '[경향포토]',
                 '[리빙포인트]', '[머니S포토]', '[부고]', '[부음]', '[사람 人 사람]',
                 '[사진]', '[서울포토]', '[영어]', "[오래전 '이날']", '[오마이포토]',
                 '[이 시각 코스피]', '[이 시각 코스닥]', '[인사]', '[코스닥 공시]', '[코스닥(개장)]',
                 '[코스닥(마감)]', '[코스피(개장)]', '[코스피(마감)]', '[포토뉴스]', '[표]',
                 '[한경로보뉴스]', '[영상]', '[오늘의 국회 ', '[오늘의 주요일정]']

stop_title_pb = ['<부고>', '<오늘의 조간 정치뉴스>', '<인사>', '<포토>', '<코>', '<유>']

stop_content_reg_ex = [r'^동영상[\s]*뉴스']

# 매칭되면 해당 내용만 삭제하는 태그, 단어 및 정규식
stop_content_reg_ex_crop = [reg_ex['email']]

stop_content_reg_ex_ml_crop = [r'^[\s]*\【.*?\】([^=\n]*기자[\s]*=)?',
                               r'^[\s]*\[.*?\]([^=\n)]*기자[\s]*=)?',
                               r'^[\s]*\(.*?\)([^=\n]*기자[\s]*=)?',
                               r'^[ㄱ-ㅎㅏ-ㅣ가-힣]+[\s]?기자[\s]?',
                               r'^© News1.*\n',
                               '(- Copyrights )?ⓒ[^\.]*$',
                               r'\【.*?\】[\s]*$',
                               r'\[.*?\][\s]*$',
                               r'\(.*?\)[\s]*$',
                               r'[\s\.][ㄱ-ㅎㅏ-ㅣ가-힣]+[\s]?기자$']

# stop_content_reg_ex_ml_crop = [r'^[\s]*\【[^\】]*\】([^=\n]*기자[\s]*=)?',
#                                r'^[\s]*\[[^\]]*\]([^=\n)]*기자[\s]*=)?',
#                                r'^[\s]*\([^\)]*\)([^=\n]*기자[\s]*=)?',
#                                r'^[ㄱ-ㅎㅏ-ㅣ가-힣]+[\s]?기자[\s]?',
#                                r'^© News1.*\n',
#                                '(- Copyrights )?ⓒ[^\.]*$',
#                                r'\【[^\【]*\】[\s]*$',
#                                r'\[[^\[]*\][\s]*$',
#                                r'\([^\(]*\)[\s]*$',
#                                r'[\s\.][ㄱ-ㅎㅏ-ㅣ가-힣]+[\s]?기자$']
# r'^[\s]*\【[^\】]*=[^\】]*\】([^=\n]*기자[\s]*=)?'
# r'^[\s]*\[[^\]]*=[^\]]*\]([^=\n)]*기자[\s]*=)?'
# r'^[\s]*\([^\)]*=[^\)]*\)([^=\n]*기자[\s]*=)?'
# /.+\n
# r'^[]{0}[]$'.format(press_ko)'

In [4]:
def check_title_delete_article(title):
    # Square brackets tag로 시작할 경우
    p = r'^' + reg_ex['sb']
    m = re.match(p, title)
    # 매칭 안 되면 다음 정규식 검사로 넘어감.
    if m == None:
        pass
    # 매칭된 문자열이 불용어 목록에 존재하면 True 반환하여 해당 기사 삭제
    elif m[0] in stop_title_sb:
        return True
    
    # Pointy brackets tag로 시작할 경우
    p = r'^' + reg_ex['pb']
    m = re.match(p, title)
    if m == None:
#         # [Debug]
#         print('None')
        pass
    elif m[0] in stop_title_pb:
#         # [Debug]
#         print('Delete')
#         print(re.sub(p, '', title))
        return True
    
    # 매칭 되지 않거나 매칭된 문자열이 불용어 목록에 존재하지 않으면 False 반환하여 해당 기사 수집
    return False

In [5]:
# # [Debug]
# title = '<부고> 매칭 & 삭제 시 삭제 안 된 나머지 + True, 매칭은 됐으나 삭제 안 될 시 False, 매칭 안 될 시 None + False 출력!'
# check_title_delete_article(title)

In [6]:
def check_content_delete_article(content):
    # 특정 단어로 시작할 경우
    for p in stop_content_reg_ex:
        m = re.match(p, content)
        # 매칭 안 되면 다음 정규식 검사로 넘어감.
        if m == None:
#             # [Debug]
#             print('None')
            pass
        # 매칭될 경우 True 반환하여 해당 기사 삭제
        else:
#             # [Debug]
#             print('Delete')
#             print(re.sub(p, '', content))
            return True
    
    # 매칭 되지 않거나 매칭된 문자열이 불용어 목록에 존재하지 않으면 False 반환하여 해당 기사 수집
    return False

In [7]:
# # [Debug]
# content = """동영상 뉴스

# 매칭 & 삭제 시 삭제 안 된 나머지 + True, 매칭은 됐으나 삭제 안 될 시 False, 매칭 안 될 시 None + False 출력!
# """
# check_content_delete_article(content)

In [8]:
def check_content_crop_content(content):
    # Normal mode
    for p in stop_content_reg_ex_crop:
        # 매칭될 경우 매칭된 문자열 잘래낸 후 앞뒤 공백 제거
        content = re.sub(p, '', content).strip()
    
    # Multiline mode
    for p in stop_content_reg_ex_ml_crop:
        content = re.sub(p, '', content, flags=re.MULTILINE).strip()

    # 잘라낸 content 반환
    return content

In [9]:
# # [Debug]
# content = """[연합뉴스] 김하나 기자

# (연합뉴스) Multiline & non-greedy test :)

# 매칭 후 삭제 안 된 나머지만 출력!
# """
# content = check_content_crop_content(content)
# print(content)

In [10]:
# [Debug]
content = """ktj@big.com
본문111
본문222
본문333

ktj@big.co.kr

본문444
본문555
본문666

ktj@bigdata.com

본문777
본문888
본문999

ktj@bigdata.co.kr

광고111
광고222
광고333
"""
# content 중 가장 마지막 기자 이메일부터 이하 내용(광고 등) 제거
try:
    p = re.findall(r'.{20}' + reg_ex['email'], content, flags=re.DOTALL)[-1]
    email_idx = re.search(p, content).start() + 20
    content_temp = content[:email_idx].strip()
    # 이메일이 본문 앞이나 중간에 들어가지 않은 경우에만 잘린 본문 사용
    if (len(content_temp) / len(content)) > 0.6:
        content = content_temp
except:
    pass

print(content)

ktj@big.com
본문111
본문222
본문333

ktj@big.co.kr

본문444
본문555
본문666

ktj@bigdata.com

본문777
본문888
본문999


## Create DB
---

In [11]:
conn = sqlite3.connect('db/news_db.db')

cur = conn.cursor()

try:
    cur.execute("CREATE TABLE daum(a_ids TEXT primary key, \
                dates DATE, times TIME, titles TEXT, contents TEXT, \
                press TEXT, authors TEXT, sections TEXT, urls TEXT)")

    conn.commit()
except:
    pass

## Scrap URLs
---

In [12]:
def get_period(date_end_time_delta, day_period):
    date_end = datetime.date.today() - datetime.timedelta(date_end_time_delta)
    date_start = date_end - datetime.timedelta(day_period - 1)
    
    return  date_start, date_end

In [13]:
# TODO: 첫 번째 인자 기준날짜(str)로 변경 
def scrap_daum_news_urls(date_end_time_delta=1, day_period=7, page_limit=10000):
    date_start, date_end = get_period(date_end_time_delta, day_period)
    
    press = 'daum'

    # Section iteration
    for section in section_list:
        if base_urls[press][section] == '':
            continue
        else:
            print(press, section_dict[section], base_urls[press][section])
            print(date_start, ' - ', date_end)
            print('--------------------------------------------------\n')
            
            # Get section
            section_ko = section_dict[section]

            # Date iteration
            for time_delta in range(day_period):
                # Get date
                date = str(date_end - datetime.timedelta(time_delta))
                print(date)

                urls = []
                
                # Page iteration
                page = 1
                while(True):
                    if (page % 100) == 0:
                        print('{0:6,} / {1:6,}'.format(page, page_limit))

                    req_url = base_urls[press][section] + '?page=' + str(page) + '&regDate=' + date.replace('-', '')
                    while(True):
                        try:
                            resp = requests.get(req_url, headers=headers, timeout=1)
                        except:
#                             print('Timeout: retry')
                            continue
                        else:
                            break
                    html = bs(resp.text, "lxml")

                    url_html = html.select('#mArticle .tit_thumb > .link_txt')                

                    urls_len = len(urls)
                    
                    # Get url
                    for i in url_html:
                        urls.append(i.get('href'))
                        
                    urls_len_diff = len(urls) - urls_len

                    if page == page_limit:
                        break
                    elif urls_len_diff == 0:
                        print('#', page, ': end of pages.')
                        break
                    else:    
                        page += 1
                
                with open('db/urls/daum/' + press + '_' + section + '_' + date.replace('-', '') + '.pkl', 'wb') as f:
                    pickle.dump(urls, f)

            print('\n--------------------------------------------------\n\n')

In [14]:
# scrap_daum_news_urls(1, 14)

## [Debug] Scrap URLs
---

In [15]:
press = 'daum'
section = 'politics'
date = '20180823'

with open('db/urls/daum/' + press + '_' + section + '_' + date + '.pkl', 'rb') as f:
    urls = pickle.load(f)
    
len(urls)

2670

In [16]:
urls[:10]

['http://v.media.daum.net/v/20180823235953274',
 'http://v.media.daum.net/v/20180823235712261',
 'http://v.media.daum.net/v/20180823234956198',
 'http://v.media.daum.net/v/20180823234929195',
 'http://v.media.daum.net/v/20180823234832186',
 'http://v.media.daum.net/v/20180823225456524',
 'http://v.media.daum.net/v/20180823224023322',
 'http://v.media.daum.net/v/20180823223902308',
 'http://v.media.daum.net/v/20180823223627276',
 'http://v.media.daum.net/v/20180823223304232']

## Scrap News
---

In [17]:
def scrap_daum_news(date_end_time_delta=1, day_period=7):
    date_start, date_end = get_period(date_end_time_delta, day_period)
    
    press = 'daum'

    for section in section_list:
        if base_urls[press][section] == '':
            continue
        else:
            print(press, section_dict[section], base_urls[press][section])
            print(date_start, ' - ', date_end)
            print('--------------------------------------------------\n')
            
            # Get section
            section_ko = section_dict[section]

            # Date iteration
            for time_delta in range(day_period):
                # Get date
                date = str(date_end - datetime.timedelta(time_delta))
                print(date)

                # Open url
                with open('db/urls/daum/' + press + '_' + section + '_' + date.replace('-', '') + '.pkl', 'rb') as f:
                    urls = pickle.load(f)
                    
                for idx, url in enumerate(urls):
                    if (idx % 1000) == 0:
                        print('{0:6,} / {1:6,}'.format(idx, len(urls)))

                    while(True):
                        try:
                            resp = requests.get(url, headers=headers, timeout=1)
#                             time_module.sleep(0.5)
                        except:
#                             print('Timeout: retry')
                            continue
                        else:
                            break
                    html = bs(resp.text, "lxml")

                    # Get id
                    a_id = 'da_' + url[-17:]
                    
                    # Get time
                    date_html = html.select('.head_view .info_view')
#                         date = re.search(r'\d{2,4}[.-]?\d+[.-]?.\d+', date_html[0].text)[0]
                    try:
                        time = re.search(r'\d+:\d+', date_html[0].text)[0]
                    except:
                        time = None
                
                    # Get title
                    # title이 빈 문자열이면 해당 기사 제외
                    title_html = html.select('.head_view .tit_view')
                    try:
                        title = title_html[0].text.strip()
                    except:
                        continue

                    # Check title
                    # title에 한글이 하나도 없으면 해당 기사 제외
                    if re.search(r'[ㄱ-ㅎㅏ-ㅣ가-힣]+', title) == None:
                        continue
                    # 특정 title의 기사 제외
                    if check_title_delete_article(title):
                        continue
                    
                    # Get content
                    # content가 빈 문자열이면 해당 기사 제외
                    content_html = html.select('.news_view #harmonyContainer')
                    try:
                        content = content_html[0].text.strip()
                    except:
                        continue
                        
                    # Check content
                    # content 중 가장 마지막 기자 이메일부터 이하 내용(광고 등) 제거
                    try:
                        p = re.findall(r'.{20}' + reg_ex['email'], content, flags=re.DOTALL)[-1]
                        email_idx = re.search(p, content).start() + 20
                        content_temp = content[:email_idx].strip()
                        # 이메일이 본문 앞이나 중간에 들어가지 않은 경우에만 잘린 본문 사용
                        if (len(content_temp) / len(content)) > 0.6:
                            content = content_temp
                    except:
                        pass
                    # 특정 content의 기사 제외
                    if check_content_delete_article(content):
                        continue
                    # content 중 특정 내용 제거
                    content = check_content_crop_content(content)
                        
                    # TODO: 예외 때문에 다른 방식을 도입하는 것이 좋을 듯하여 제거 예정
                    # content 중 ▶ 이하 내용 제거 (naver에 적용했던 코드이므로 수정해서 사용할 것)
#                         tempIdx = cont.index('▶')
#                         if tempIdx > 0:
#                             cont = cont[:tempIdx]

                    # Get author
                    # TODO: 정규식 수정
                    try:
                        author = re.search(r'\w+\s*기자', content_html[0].text)[0].replace(' 기자', '').replace('기자', '')
                    except:
                        author = None

                    # Get press
                    press_html = html.select('.head_view .thumb_g')
                    try:
                        press_ko = press_html[0].get('alt')
                    except:
                        press_ko = None

                    # Insert data (row by row)
                    data = (a_id, date, time, title, content, press_ko, author, section_ko, url)
                    try:
                        cur.execute("INSERT INTO daum(a_ids, dates, times, titles, contents, press, authors, sections, urls) \
                                    values(?,?,?,?,?,?,?,?,?)", data)
                    except:
                        pass
                    else:
                        conn.commit()

            print('\n--------------------------------------------------\n\n')

In [ ]:
scrap_daum_news(2, 1)

daum 사회 http://media.daum.net/breakingnews/society
2018-08-23  -  2018-08-23
--------------------------------------------------

2018-08-23
     0 /  7,994
 1,000 /  7,994
 2,000 /  7,994


In [ ]:
# # 에러 발생 시 DB의 특정 section data 삭제
# conn = sqlite3.connect('db/news_db.db')
# cur = conn.cursor()

# cur.execute("DELETE FROM daum WHERE sections='IT'")

# conn.commit()